In [14]:
import pandas as pd
import numpy as np

# Graph
import plotly.express as px
import matplotlib.pyplot as plt

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras import callbacks

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# tf
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import SGD

In [2]:
path = r"C:\Users\sonia\Documents\Document\2022-2023\Projet 3I - Deep Learning\dataset_clean.csv"
TARGET = 'Label'

In [3]:
df = pd.read_csv(path)

In [4]:
df = df[['Init_Win_bytes_forward',
'Total Length of Fwd Packets',
'Bwd Header Length',
'Destination Port',
'Subflow Fwd Bytes',
'Packet Length Std',
'Packet Length Variance',
'Bwd Packets/s',
'Average Packet Size',
'Bwd Packet Length Std',
         "Label"]]

In [5]:
df_b = df[df['Label']==1]
df_a = df[df['Label']==0]

In [6]:
df_b= df_b[0:2241]

In [7]:
df = pd.concat([df_a, df_b], axis=0)

In [8]:
df = df.sample(frac=1)

In [9]:
y = df[TARGET]
X = df.drop([TARGET],axis=1)

In [10]:
y = y.to_numpy()
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
y = to_categorical(y)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [12]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X.shape[1]))

## Model

In [15]:
model = Sequential()

model.add(GRU(220, input_shape=(1,10), activation="relu", return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(220,activation="relu"))
model.add(Dense(y.shape[1], activation='sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 1, 220)            153120    
                                                                 
 dropout (Dropout)           (None, 1, 220)            0         
                                                                 
 gru_1 (GRU)                 (None, 220)               291720    
                                                                 
 dense (Dense)               (None, 2)                 442       
                                                                 
Total params: 445,282
Trainable params: 445,282
Non-trainable params: 0
_________________________________________________________________


In [22]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", 
                                        patience = 20, 
                                        restore_best_weights = True)

In [23]:
history = model.fit(X_train, y_train, 
                    epochs = 200, 
                    validation_data= (X_test, y_test),
                    callbacks =[earlystopping])
history

Epoch 1/200
113/113 [==============================] - 1s 8ms/step - loss: 0.1608 - accuracy: 0.9308 - val_loss: 0.1976 - val_accuracy: 0.9186
Epoch 2/200
113/113 [==============================] - 1s 8ms/step - loss: 0.1578 - accuracy: 0.9331 - val_loss: 0.2003 - val_accuracy: 0.9197
Epoch 3/200
113/113 [==============================] - 1s 8ms/step - loss: 0.1630 - accuracy: 0.9319 - val_loss: 0.1929 - val_accuracy: 0.9197
Epoch 4/200
113/113 [==============================] - 1s 8ms/step - loss: 0.1596 - accuracy: 0.9291 - val_loss: 0.2140 - val_accuracy: 0.9108
Epoch 5/200
113/113 [==============================] - 1s 8ms/step - loss: 0.1674 - accuracy: 0.9269 - val_loss: 0.1981 - val_accuracy: 0.9153
Epoch 6/200
113/113 [==============================] - 1s 9ms/step - loss: 0.1638 - accuracy: 0.9280 - val_loss: 0.2076 - val_accuracy: 0.9086
Epoch 7/200
113/113 [==============================] - 1s 8ms/step - loss: 0.1588 - accuracy: 0.9300 - val_loss: 0.2020 - val_accuracy: 0.8885

In [24]:
accuracy = model.evaluate(X_test, y_test)
accuracy # 0.9175

29/29 [==============================] - 0s 3ms/step - loss: 0.1885 - accuracy: 0.9175


[0.1884540468454361, 0.917502760887146]